# Instructions

There are four parts to the codes below:
- Main
- Links Master List
- URLLIB Web Scraper
- Selenium Plot Scraper

Steps:
- Run Main once for a given city
- Run Links Master List once for a given city

- Run URLLIB Web Scraper as many times as it takes to scrape all the links as your scraping may stop due to a captcha (check how many links are left inside the Proj-masterlist-(your city) csv)

- Run Selenium Plot Scraper as many times as it takes to scrape all the links as your scraping may stop due to a captcha (check how many links are left inside the Plot-masterlist-(your city) csv)




# Main 

contains all the all packages and variables needed for all methods to run

install selenium and chromedriver in order to use some of the below packages

import urllib.request 
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS

-- For ignoring SSL certificate errors -- 
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = "https://www.zillow.com/homedetails/1998-Pacific-Ave-APT-204-San-Francisco-CA-94109/15073046_zpid/"

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

soup = BS(webpage,'html.parser')
h3=soup.findAll('div', attrs={'class':'ds-overview-section'})[1].text
h3


In [1077]:
# main
import urllib.request 
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import time
import sys
import numpy as np
import pandas as pd
import requests
import re
from selenium import webdriver as web
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
from datetime import date


# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

city = 'foster-city'
BayCity_url = "https://www.zillow.com/homes/for_sale/" + city + "_rb/"
listing_links=[]
total=[]

chromeDriverDir = '/Users/dkwliu/Desktop/BAN_612/python/chromedriver'

pagesToScrape = 2

# Links Master List

- creates a list of links for for-sale houses in a specified city

In [1078]:
# Function to get the links from zillow

def get_house_links(url, pages):
    house_links=[]
    
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    for i in range(pages):
        p=i+2
        soup = BS(webpage, 'html.parser')
        listings = soup.find_all("a", class_="list-card-link list-card-img")
        house_links=[row['href'] for row in listings]
        total.append(house_links)
        
        # finding link to next page
      
        next_button_link = str('https://www.zillow.com/homes/for_sale/' + city + '/' +str(p)+'_p/') 
        if i<19:
            url=next_button_link
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()
    
    return total


In [1079]:
# Convert list of lists of links into 1 Master List of all property listings.
def flatten_list(house_links_All_pages):
    house_links_MASTER = []
    for sublist in house_links_All_pages:
        for item in sublist:
            house_links_MASTER.append(item)
    return house_links_MASTER    

In [1080]:
house_links_All_pages = get_house_links(BayCity_url,pages=pagesToScrape) 

In [1081]:
#house_links_All_pages

In [1082]:
# flattens list of links into a single list
house_links_MASTER = flatten_list(house_links_All_pages)
#house_links_MASTER

In [1083]:
# save the links into an excel file
house_links_MASTER_DF = pd.DataFrame(house_links_MASTER, columns=['Links']) 
masterlistName = "masterlist-" + city + ".csv"
house_links_MASTER_DF.to_csv(masterlistName, index = False, encoding = "UTF-8")
house_links_MASTER_DF.to_csv('Proj-' + masterlistName, index = False, encoding = "UTF-8")
house_links_MASTER_DF.to_csv('Plot-' + masterlistName, index = False, encoding = "UTF-8")

# URLLIB Web Scraper

In [1084]:
# Functions to scrape separate fields

# Property's header info
def get_price(soup):
    try:
        p=soup.findAll('span',attrs={'class':'ds-value'} )
        price = p[0].get_text().replace(",", "").replace("+", "").replace("$", "").lower()
        return int(price)
    except:
        return np.nan

def get_address(soup):
    try:
        h1=soup.find('h1', attrs={'class':'ds-address-container'})
        address=h1.find_all('span')[0].text.strip()
        return address
    except:
        return 'None'
def get_city(soup):
    try:
        h1=soup.find('h1', attrs={'class':'ds-address-container'})
        city=h1.find_all('span')[1].text.split(',')[0]
        return city
    except:
        return 'None'

def get_state(soup):
    try:
        h1=soup.find('h1', attrs={'class':'ds-address-container'})
        state=h1.find_all('span')[1].text.split(',')[1].split(' ')[1]
        return state
    except:
        return 'None'    

def get_zip(soup):
    try:
        h1=soup.find('h1', attrs={'class':'ds-address-container'})
        zip_code=h1.find_all('span')[1].text.split(',')[1].split(' ')[2]
        return zip_code
    except:
        return 'None'
    
def get_num_beds(soup):
    try:
        h3=soup.find('h3', attrs={'class':'ds-bed-bath-living-area-container'})
        beds=h3.find_all('span', attrs={'class':'ds-bed-bath-living-area'})[0].text.split(' ')[0]
        return beds
    except:
        return 'None'
    
def get_num_baths(soup):
    try:
        #bath = re.findall('<span>(.*?)</span><span class="ds-summary-row-label-secondary"> <!-- -->ba</span>', str(soup))[0]
        h3=soup.find('h3', attrs={'class':'ds-bed-bath-living-area-container'})
        bath=h3.find_all('span', attrs={'class':'ds-bed-bath-living-area'})[1].text.split(' ')[0]
        #bath = re.findall('[0-9]+',str(h3))[0]
        return bath
    except:
        return 'None'
    
def get_floor_size(soup):
    try:
        h3=soup.find('h3', attrs={'class':'ds-bed-bath-living-area-container'})
        floor_size=h3.find_all('span', attrs={'class':'ds-bed-bath-living-area'})[2].text.split(' ')[0]
        return floor_size
    except:
        return 'None'
    
    
    
# Property's facts and features section   
def get_year_built(soup):
    try:
        year_built = re.findall('<span class="ds-standard-label ds-home-fact-label">Year built:</span><span class="ds-body ds-home-fact-value">(.*?)</span>',str(soup))[0]
        return year_built
    except:
        return 'None'


def get_Type(soup):
    try:
        Type = re.findall('<span class="ds-standard-label ds-home-fact-label">Type:</span><span class="ds-body ds-home-fact-value">(.*?)</span>',str(soup))[0]
        return Type
    except:
        return 'None'

def get_Parking(soup):
    try:
        Parking = re.findall('<span class="ds-standard-label ds-home-fact-label">Parking:</span><span class="ds-body ds-home-fact-value">(.*?)</span>',str(soup))[0]
        return Parking
    except:
        return 'None'
    
def get_HOA(soup):
    try:
        HOA = re.findall('<span class="ds-standard-label ds-home-fact-label">HOA:</span><span class="ds-body ds-home-fact-value">(.*?)</span>',str(soup))[0]
        return HOA
    except:
        return 'None'
    
def get_Lot(soup):
    try:
        Lot = re.findall('<span class="ds-standard-label ds-home-fact-label">Lot:</span><span class="ds-body ds-home-fact-value">(.*?)</span>',str(soup))[0]
        return Lot
    except:
        return 'None'
    
def get_PricePerSqFt(soup):
    try:
        PricePerSqFt = re.findall('<span class="ds-standard-label ds-home-fact-label">Price/sqft:</span><span class="ds-body ds-home-fact-value">(.*?)</span>',str(soup))[0]
        return PricePerSqFt
    except:
        return 'None'
    
def get_Description(soup):
    try:
        Description=soup.findAll('div', attrs={'class':'ds-overview-section'})[1].text
        return Description
    except:
        return 'None'
    
def get_remodel_year(soup):
    try:
        remodel_year = re.findall('<span class="Text-aiai24-0 IJYzV">Major remodel year: <!-- -->(.*?)</span>',str(soup))[0]
        return remodel_year
    except:
        return 'None'
    
def get_Schools(soup):
    try:
        School_List=[]
        Schools=soup.findAll('a', attrs={'class':'ds-school-name ds-standard-label notranslate'})
        i=0
        while i<3:
            
            Schools=soup.findAll('a', attrs={'class':'ds-school-name ds-standard-label notranslate'})[i].text
            School_List.append(Schools)
            i+=1
        return School_List
    except:
        return 'None'    

In [1085]:
import pandas as pd
def get_house_data(house_links_MASTER):
    house_data = []
    for link in house_links_MASTER:
        
        urlString = link.split('/')
        
        if urlString[3] == 'homedetails': 
            print('scraping', link)
            req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req).read()
            soup = BS(webpage,'html.parser')

            address=get_address(soup)

            if address != 'None':
                price=get_price(soup)
                city=get_city(soup)
                state=get_state(soup)
                zipcode=get_zip(soup)
                beds=get_num_beds(soup)
                baths=get_num_baths(soup)
                floor_size=get_floor_size(soup)
                year_built=get_year_built(soup)
                Property_Type=get_Type(soup)
                Parking=get_Parking(soup)
                HOA=get_HOA(soup)
                Lot=get_Lot(soup)
                PricePerSqFt=get_PricePerSqFt(soup)
                Description=get_Description(soup)
                remodel_year = get_remodel_year(soup)
                Schools=get_Schools(soup)

                house_data.append([price, address, city, state, zipcode, beds, baths, floor_size, year_built, Property_Type, HOA, Lot, PricePerSqFt, Description, remodel_year, Schools])
            else:
                print('ending scrape')
                break
        else:
            address = 'None'
            price='None'
            city='None'
            state='None'
            zipcode='None'
            beds='None'
            baths='None'
            floor_size='None'
            year_built='None'
            Property_Type='None'
            Parking='None'
            HOA='None'
            Lot='None'
            PricePerSqFt='None'
            Description='None'
            remodel_year ='None'
            Schools = 'None'
            
            house_data.append([price, address, city, state, zipcode, beds, baths, floor_size, year_built, Property_Type, HOA, Lot, PricePerSqFt, Description, remodel_year, Schools])
        
        #house_data={'price':[price], 'address':[address], 'city':[city], 'state':[state], 'zipcode':[zipcode], 'beds':[beds], 'baths':[baths], 'floor_size':[floor_size], 'year_built':[year_built], 'Property_Type':[Property_Type], 'HOA':[HOA], 'PricePerSqFt':[PricePerSqFt], 'Description':[Description]}
        #house_data_df=pd.DataFrame.from_dict(house_data)
    return house_data

In [1086]:
#Utilize the copy of the masterlist specifically for the project web scraper to work on
try:
    Proj_masterlist_df = pd.read_csv('Proj-masterlist-' + city + '.csv')
    Proj_masterlist_df_rowcount = Proj_masterlist_df.iloc[:,:].count()[0]
    Proj_links_list = []
    house_data_final = pd.DataFrame()
    
    if Proj_masterlist_df_rowcount > 0:
        Proj_links_rows = Proj_masterlist_df.values.tolist()
        for each in Proj_links_rows:
            Proj_links_list.append(each[0])
            
        print('copying over houses')
        house_data_final = get_house_data(Proj_links_list)
        print('Finished')
    else:
        print("There are no links found for this city")
except:
    print("There are no links left for this city")

copying over houses
scraping https://www.zillow.com/homedetails/620-Mystic-Ln-Foster-City-CA-94404/15615660_zpid/
scraping https://www.zillow.com/homedetails/1041-Shell-Blvd-APT-7-Foster-City-CA-94404/15627410_zpid/
scraping https://www.zillow.com/homedetails/607-Pisces-Ln-Foster-City-CA-94404/15613602_zpid/
scraping https://www.zillow.com/homedetails/753-Lurline-Dr-Foster-City-CA-94404/15611047_zpid/
scraping https://www.zillow.com/homedetails/1263-Martinique-Ln-Foster-City-CA-94404/15615576_zpid/
scraping https://www.zillow.com/homedetails/1869-Beach-Park-Blvd-Foster-City-CA-94404/15611529_zpid/
scraping https://www.zillow.com/homedetails/1050-Helm-Ln-Foster-City-CA-94404/15619324_zpid/
scraping https://www.zillow.com/homedetails/20-Charthouse-Ln-Foster-City-CA-94404/15619169_zpid/
scraping https://www.zillow.com/homedetails/58-Port-Royal-Ave-Foster-City-CA-94404/15616509_zpid/
scraping https://www.zillow.com/homedetails/945-Lido-Ln-Foster-City-CA-94404/2080759202_zpid/
scraping http

In [1087]:
columns = ["price", "address", "city", "state", "zipcode", "beds", "baths", "floor_size", "year_built", "Property_Type", "HOA", "Lot", "PricePerSqFt", "Description", "remodel_year", 'Schools']
pd.DataFrame(house_data_final, columns = columns)


,price,address,city,state,zipcode,beds,baths,floor_size,year_built,Property_Type,HOA,Lot,PricePerSqFt,Description,remodel_year,Schools
0,1788000,"620 Mystic Ln,",Foster City,CA,94404,3,3,"1,940",1977,Single Family,$273/month,"5,279 sqft",$922,"Open HouseSat, Mar 71:30 PM - 4:30 PM{""@type"":...",None,"[Foster City Elementary School, Bowditch Middl..."
1,754000,"1041 Shell Blvd APT 7,",Foster City,CA,94404,2,2,"1,042",1972,Condo,$530/month,None,$724,None AvailableRead more,1972,"[Foster City Elementary School, Bowditch Middl..."
2,1275000,"607 Pisces Ln,",Foster City,CA,94404,3,2,"1,470",1973,Townhouse,$395/month,"2,922 sqft",$867,This fabulously remodeled end-unit is mere ste...,None,"[Foster City Elementary School, Bowditch Middl..."
3,2350000,"753 Lurline Dr,",Foster City,CA,94404,5,3,"2,530",1965,Single Family,None,0.36 acres,$929,"Open HousesSat, Mar 71:00 PM - 4:00 PM{""@type""...",None,"[Brewer Island Elementary School, Bowditch Mid..."
4,1788000,"1263 Martinique Ln,",Foster City,CA,94404,3,3,"2,010",1976,Single Family,$228/month,"4,599 sqft",$890,"Open HousesSat, Mar 71:30 PM - 4:30 PM{""@type""...",None,"[Foster City Elementary School, Bowditch Middl..."
5,1549000,"1869 Beach Park Blvd,",Foster City,CA,94404,3,2,"1,890",1965,Single Family,None,"5,702 sqft",$820,"Open HouseSun, Mar 151:00 PM - 4:00 PM{""@type""...",None,"[Audubon Elementary School, Bowditch Middle Sc..."
6,1324370,"1050 Helm Ln,",Foster City,CA,94404,3,2.5,"1,530",1989,Single Family,$455/month,871 sqft,None,Buy this foreclosure deal the right way. Visit...,None,"[Audubon Elementary School, Bowditch Middle Sc..."
7,1599000,"20 Charthouse Ln,",Foster City,CA,94404,3,3,"1,930",1988,Townhouse,$409/month,"1,999 sqft",$828,What I love about this homeArguably the most s...,2009,"[Foster City Elementary School, Bowditch Middl..."
8,1868000,"58 Port Royal Ave,",Foster City,CA,94404,3,3,"1,920",1985,Single Family,$121/month,"6,825 sqft",$973,Enjoy the wonderful Bay Area Lifestyle in this...,2005,"[Foster City Elementary School, Bowditch Middl..."
9,1249888,"945 Lido Ln,",Foster City,CA,94404,2,2,"1,440",1975,Condo,$593/month,None,$868,Beautifully remodeled waterfront condo in the ...,None,"[Foster City Elementary School, Bowditch Middl..."


In [1088]:
# Transfering data to a csv (list of lists -> dataframe -> csv)

columns = ["price", "address", "city", "state", "zipcode", "beds", "baths", "floor_size", "year_built", "Property_Type", "HOA", "Lot", "PricePerSqFt", "Description", "remodel_year", 'Schools']
house_data_df=pd.DataFrame(house_data_final, columns = columns)
csvprjname = 'ZillowProjData-' + city + '.csv'

scraped_rows_count = len(house_data_df)

# Remaining list that hasn't been scraped due to recaptcha
if scraped_rows_count > 0:
    print(scraped_rows_count)
    new_Proj_links_list = pd.DataFrame(Proj_links_list[scraped_rows_count:])
    
    if len(new_Proj_links_list) > 0:
        new_Proj_links_list.to_csv('Proj-masterlist-' + city + '.csv', index=False, encoding="UTF-8")
    else:
        print('deleting old masterlist')
        os.remove('Proj-masterlist-' + city + '.csv')
        
    try:
        prevDF = pd.read_csv(csvprjname)
        prevDF = pd.concat([prevDF,house_data_df], ignore_index=True, sort=False)
        print("adding to a previous csv file")
        prevDF.to_csv(csvprjname, index=False, encoding = 'UTF-8')
    except:
        print('creating a new csv file')
        house_data_df.to_csv(csvprjname, index = False, encoding = "UTF-8")

64
deleting old masterlist
creating a new csv file


In [1089]:
# gets rid of all rows with 'None' for its address
existing_CSV = pd.read_csv(csvprjname)
editted_CSV = existing_CSV.loc[existing_CSV['address'] != 'None']
editted_CSV = editted_CSV.fillna('None')
editted_CSV.to_csv(csvprjname, index = False, encoding = "UTF-8")

# Selenium Plot Scraper

In [775]:
# creates dataframe and csv for plot data

def plotData(links, chromedrvr):

    today = date.today()
    todayStr = str(today).split('-')
    todayYrStr = todayStr[0]
    
    months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    dateList = ['address','city','state','zipcode']
    
    for yr in range(12):
        for m in months:
            tdate = m + " " + str(int(todayYrStr)-10+yr)      
            dateList.append(tdate)
            
    plotDataDF = pd.DataFrame(columns=dateList)

    print("DF set")

    house_links = links
    
    # change directory below to indicate where your chrome driver is located
    driver = chromedrvr

    try:
        for each in house_links:
            urlString = each.split('/')
            
            if urlString[3] == 'homedetails':
                print('good url')
                plotDataDF = pd.concat([plotDataDF, plotScrape(each, driver)],sort=False, ignore_index=True)
                print(plotDataDF)  
            else:
                print('bad url')
                plotDataDF = pd.concat([plotDataDF, pd.DataFrame([['None']], columns=['address'])],sort=False, ignore_index=True)
                print(plotDataDF)
                
        print('Finished. All links in masterlist were scraped.')
    except:
        print('Finished. Some links from masterlist still remains.')
    
    return plotDataDF


In [776]:
# method for scraping the plot data
def plotScrape(link, d):
    prev_price_date = ""
    this_home_price = ""
    x = 0

    d.get(link)

    colNames = []
    recordData = [[]]
    colNames.append('address')
    colNames.append('city')
    colNames.append('state')
    colNames.append('zipcode')
    address = d.find_element_by_xpath('//h1[@class="ds-address-container"]').text
    addressParsed = address.split(', ')
    addressStr = ''

    if len(addressParsed[:len(addressParsed) - 2]) > 1:
        for each in addressParsed[:len(addressParsed) - 2]:
            addressStr += each + ', '
        addressStr = addressStr[:len(addressStr)-1]    
    else:
        addressStr = addressParsed[:len(addressParsed) - 2][0] + ','
    
    recordData[0].append(addressStr)
    recordData[0].append(addressParsed[len(addressParsed) - 2])
    recordData[0].append(addressParsed[len(addressParsed) - 1].split(' ')[0])
    recordData[0].append(addressParsed[len(addressParsed) - 1].split(' ')[1])
    
    stopScrape = False

    print(address)

    #stopScrape variable indicates when to stop scraping the plot
    while stopScrape == False:
        try:
            element_to_hover_over = d.find_element_by_xpath('//*[@id="ds-home-values"]/div/div[1]/div/div[4]/div')
            hover = ActionChains(d).move_to_element_with_offset(element_to_hover_over,x,200)
            hover.perform()

            selected_date = d.find_element_by_xpath('//*[@id="ds-home-values"]/div/div[1]/div/div[4]/div/div[2]/div/div').text

            if selected_date == "Sold for" or selected_date == "Listed for":

                strDate = d.find_element_by_xpath('//*[@id="ds-home-values"]/div/div[1]/div/div[4]/div/div[2]/div/div[3]').text

                strDateParsed = strDate.split(' ')[1].split('/')

                if strDateParsed[0] == "1":
                    selected_date = "Jan " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "2":
                    selected_date = "Feb " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "3":
                    selected_date = "Mar " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "4":
                    selected_date = "Apr " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "5":
                    selected_date = "May " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "6":
                    selected_date = "Jun " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "7":
                    selected_date = "Jul " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "8":
                    selected_date = "Aug " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "9":
                    selected_date = "Sep " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "10":
                    selected_date = "Oct " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "11":
                    selected_date = "Nov " + "20" + strDateParsed[2]
                elif strDateParsed[0] == "12":
                    selected_date = "Dec " + "20" + strDateParsed[2]

            if selected_date != prev_price_date:  
                colNames.append(selected_date)
                prev_price_date = selected_date

                this_home_price = d.find_element_by_xpath('//*[@id="ds-home-values"]/div/div[1]/div/div[4]/div/div[1]/div[1]/div[2]/span').text
                recordData[0].append(this_home_price)

            x += 3
        except:
            stopScrape = True

    recordDF = pd.DataFrame(recordData, columns=colNames)   
    return recordDF


In [777]:
#Utilize the copy of the masterlist specifically for the selenium plot web scraper to work on

try:
    Plot_masterlist_df = pd.read_csv('Plot-masterlist-' + city + '.csv')
    Plot_masterlist_df_rowcount = Plot_masterlist_df.iloc[:,:].count()[0]
    Plot_links_list = []
    plotDF = pd.DataFrame()
    
    if Plot_masterlist_df_rowcount > 0:
        Plot_links_rows = Plot_masterlist_df.values.tolist()
        for each in Plot_links_rows:
            Plot_links_list.append(each[0])

        chromeDriver = web.Chrome(chromeDriverDir)
        plotDF = plotData(Plot_links_list,chromeDriver)
    else:
        print("There are no links remaining for this city")
except:
    print("There are no links left for this city")


DF set
good url
['38563 Royal Ann Cmn', 'Fremont', 'CA 94536']
38563 Royal Ann Cmn, Fremont, CA 94536
                address     city state zipcode Jan 2010 Feb 2010 Mar 2010  \
0  38563 Royal Ann Cmn,  Fremont    CA   94536      NaN      NaN      NaN   

  Apr 2010 May 2010 Jun 2010  ... Mar 2021 Apr 2021 May 2021 Jun 2021  \
0      NaN      NaN  $285.5K  ...  $588.1K      NaN      NaN      NaN   

  Jul 2021 Aug 2021 Sep 2021 Oct 2021 Nov 2021 Dec 2021  
0      NaN      NaN      NaN      NaN      NaN      NaN  

[1 rows x 148 columns]
good url
['3671 Lake Ontario Dr', 'Fremont', 'CA 94555']
3671 Lake Ontario Dr, Fremont, CA 94555
                 address     city state zipcode Jan 2010 Feb 2010 Mar 2010  \
0   38563 Royal Ann Cmn,  Fremont    CA   94536      NaN      NaN      NaN   
1  3671 Lake Ontario Dr,  Fremont    CA   94555      NaN      NaN      NaN   

  Apr 2010 May 2010 Jun 2010  ... Mar 2021 Apr 2021 May 2021 Jun 2021  \
0      NaN      NaN  $285.5K  ...  $588.1K      NaN

['40346 Blanchard St', 'Fremont', 'CA 94538']
40346 Blanchard St, Fremont, CA 94538
                       address     city state zipcode Jan 2010 Feb 2010  \
0         38563 Royal Ann Cmn,  Fremont    CA   94536      NaN      NaN   
1        3671 Lake Ontario Dr,  Fremont    CA   94555      NaN      NaN   
2              2713 Pismo Ter,  Fremont    CA   94538      NaN      NaN   
3          34203 Aberdeen Ter,  Fremont    CA   94555      NaN      NaN   
4                88 Delta Grn,  Fremont    CA   94538      NaN      NaN   
5             5587 Via Lugano,  Fremont    CA   94555      NaN      NaN   
6  4141 Deep Creek Rd SPC 193,  Fremont    CA   94555      NaN      NaN   
7          40346 Blanchard St,  Fremont    CA   94538      NaN      NaN   

  Mar 2010 Apr 2010 May 2010 Jun 2010  ... Mar 2021 Apr 2021 May 2021  \
0      NaN      NaN      NaN  $285.5K  ...  $588.1K      NaN      NaN   
1      NaN      NaN      NaN  $536.5K  ...    $1.1M      NaN      NaN   
2      NaN      NaN  

['39078 Guardino Dr APT 107', 'Fremont', 'CA 94538']
39078 Guardino Dr APT 107, Fremont, CA 94538
                        address     city state zipcode Jan 2010 Feb 2010  \
0          38563 Royal Ann Cmn,  Fremont    CA   94536      NaN      NaN   
1         3671 Lake Ontario Dr,  Fremont    CA   94555      NaN      NaN   
2               2713 Pismo Ter,  Fremont    CA   94538      NaN      NaN   
3           34203 Aberdeen Ter,  Fremont    CA   94555      NaN      NaN   
4                 88 Delta Grn,  Fremont    CA   94538      NaN      NaN   
5              5587 Via Lugano,  Fremont    CA   94555      NaN      NaN   
6   4141 Deep Creek Rd SPC 193,  Fremont    CA   94555      NaN      NaN   
7           40346 Blanchard St,  Fremont    CA   94538      NaN      NaN   
8               5315 Brophy Dr,  Fremont    CA   94536      NaN      NaN   
9              3340 Hogarth Pl,  Fremont    CA   94555      NaN      NaN   
10              34935 Nova Ter,  Fremont    CA   94555      NaN   

['4936 Creekwood Dr', 'Fremont', 'CA 94555']
4936 Creekwood Dr, Fremont, CA 94555
                        address     city state zipcode Jan 2010 Feb 2010  \
0          38563 Royal Ann Cmn,  Fremont    CA   94536      NaN      NaN   
1         3671 Lake Ontario Dr,  Fremont    CA   94555      NaN      NaN   
2               2713 Pismo Ter,  Fremont    CA   94538      NaN      NaN   
3           34203 Aberdeen Ter,  Fremont    CA   94555      NaN      NaN   
4                 88 Delta Grn,  Fremont    CA   94538      NaN      NaN   
5              5587 Via Lugano,  Fremont    CA   94555      NaN      NaN   
6   4141 Deep Creek Rd SPC 193,  Fremont    CA   94555      NaN      NaN   
7           40346 Blanchard St,  Fremont    CA   94538      NaN      NaN   
8               5315 Brophy Dr,  Fremont    CA   94536      NaN      NaN   
9              3340 Hogarth Pl,  Fremont    CA   94555      NaN      NaN   
10              34935 Nova Ter,  Fremont    CA   94555      NaN      NaN   
11   3

['4560 Val St', 'Fremont', 'CA 94538']
4560 Val St, Fremont, CA 94538
                        address     city state zipcode Jan 2010 Feb 2010  \
0          38563 Royal Ann Cmn,  Fremont    CA   94536      NaN      NaN   
1         3671 Lake Ontario Dr,  Fremont    CA   94555      NaN      NaN   
2               2713 Pismo Ter,  Fremont    CA   94538      NaN      NaN   
3           34203 Aberdeen Ter,  Fremont    CA   94555      NaN      NaN   
4                 88 Delta Grn,  Fremont    CA   94538      NaN      NaN   
5              5587 Via Lugano,  Fremont    CA   94555      NaN      NaN   
6   4141 Deep Creek Rd SPC 193,  Fremont    CA   94555      NaN      NaN   
7           40346 Blanchard St,  Fremont    CA   94538      NaN      NaN   
8               5315 Brophy Dr,  Fremont    CA   94536      NaN      NaN   
9              3340 Hogarth Pl,  Fremont    CA   94555      NaN      NaN   
10              34935 Nova Ter,  Fremont    CA   94555      NaN      NaN   
11   39078 Guardin

['3371 Baywood Ter APT 113', 'Fremont', 'CA 94536']
3371 Baywood Ter APT 113, Fremont, CA 94536
                        address     city state zipcode Jan 2010 Feb 2010  \
0          38563 Royal Ann Cmn,  Fremont    CA   94536      NaN      NaN   
1         3671 Lake Ontario Dr,  Fremont    CA   94555      NaN      NaN   
2               2713 Pismo Ter,  Fremont    CA   94538      NaN      NaN   
3           34203 Aberdeen Ter,  Fremont    CA   94555      NaN      NaN   
4                 88 Delta Grn,  Fremont    CA   94538      NaN      NaN   
5              5587 Via Lugano,  Fremont    CA   94555      NaN      NaN   
6   4141 Deep Creek Rd SPC 193,  Fremont    CA   94555      NaN      NaN   
7           40346 Blanchard St,  Fremont    CA   94538      NaN      NaN   
8               5315 Brophy Dr,  Fremont    CA   94536      NaN      NaN   
9              3340 Hogarth Pl,  Fremont    CA   94555      NaN      NaN   
10              34935 Nova Ter,  Fremont    CA   94555      NaN     

['4452 Ogden Dr', 'Fremont', 'CA 94538']
4452 Ogden Dr, Fremont, CA 94538
                        address     city state zipcode Jan 2010 Feb 2010  \
0          38563 Royal Ann Cmn,  Fremont    CA   94536      NaN      NaN   
1         3671 Lake Ontario Dr,  Fremont    CA   94555      NaN      NaN   
2               2713 Pismo Ter,  Fremont    CA   94538      NaN      NaN   
3           34203 Aberdeen Ter,  Fremont    CA   94555      NaN      NaN   
4                 88 Delta Grn,  Fremont    CA   94538      NaN      NaN   
5              5587 Via Lugano,  Fremont    CA   94555      NaN      NaN   
6   4141 Deep Creek Rd SPC 193,  Fremont    CA   94555      NaN      NaN   
7           40346 Blanchard St,  Fremont    CA   94538      NaN      NaN   
8               5315 Brophy Dr,  Fremont    CA   94536      NaN      NaN   
9              3340 Hogarth Pl,  Fremont    CA   94555      NaN      NaN   
10              34935 Nova Ter,  Fremont    CA   94555      NaN      NaN   
11   39078 Gua

In [778]:
# create the csv files
csvplotname = "PlotData-" + city + ".csv"

plotdata_rows_count = len(plotDF)

# Remaining list that hasn't been scraped due to recaptcha
if plotdata_rows_count > 0:
    print(plotdata_rows_count)
    new_Plot_links_list = pd.DataFrame(Plot_links_list[plotdata_rows_count:])
    
    if len(new_Plot_links_list) > 0:
        new_Plot_links_list.to_csv('Plot-masterlist-' + city + '.csv', index=False, encoding="UTF-8")
    else:
        print('deleting old masterlist')
        os.remove('Plot-masterlist-' + city + '.csv')
    
    try:
        prevPlotDF = pd.read_csv(csvplotname)
        prevPlotDF = pd.concat([prevPlotDF,plotDF], ignore_index=True, sort=False)
        print("adding to a previous csv file")
        prevPlotDF = prevPlotDF.fillna('None')
        prevPlotDF.to_csv(csvplotname, index=False, encoding = 'UTF-8')
    except:
        print('creating a new csv file')
        plotDF = plotDF.fillna('None')
        plotDF.to_csv(csvplotname, index = False, encoding = "UTF-8")

22
adding to a previous csv file


Quit chromedriver only when you're done

In [779]:
chromeDriver.quit()

In [1090]:
# simple functions to check the difference between the links found for a city
# and the number of links that were actually scraped
current_ext_list = pd.read_csv('masterlist-'+city+'.csv')
current_ext = pd.read_csv('ZillowProjData-' +city + '.csv')
print('list ',len(current_ext_list))
print('df ', len(current_ext))

list  64
df  36


# CSV Collater
# DO NO USE ...
... until all the separate CSVs you want are in your directory

# Collate ZillowProjData CSVs

In [160]:
files = os.listdir()

zillowProjData = pd.DataFrame()

for each in files:
    parsedFile = each.split('-')
    if parsedFile[0] == 'ZillowProjData':
        csvFile = pd.read_csv(each)
        zillowProjData = pd.concat([zillowProjData, csvFile], ignore_index=True, sort=False)
        
zillowProjData.to_csv('ZillowProjData-All.csv',index=False, encoding = "UTF-8")

# Collate Plot CSVs

In [1096]:
files = os.listdir()

PlotData = pd.DataFrame()

for each in files:
    parsedFile = each.split('-')
    if parsedFile[0] == 'PlotData':
        csvFile = pd.read_csv(each)
        PlotData = pd.concat([PlotData, csvFile], ignore_index=True, sort=False)
        
PlotData.to_csv('PlotData-All.csv',index=False, encoding = "UTF-8")

# DATA PREPROCESSING

Removing Missing values and outliers

In [168]:
import pandas as pd

data=pd.read_csv('ZillowProjData-All.csv')
#data.head()

In [169]:
# deleting the rows with beds in decimals other than .5 like .1, .2, etc...(other features are mostly none here)

patternDel = '[0-9]*\.[1-4|6-9]$'
filter1 = data['beds'].str.contains(patternDel)

data=data[~filter1]
#data

In [170]:
#deleting data with beds in thousand (other features are mostly none here)
filter2 = data['beds'].str.len()>3

data=data[~filter2]
#data

In [171]:
# deleting rows where price is none
filter3 = data['price']=='None'

data=data[~filter3]
#data

In [172]:
# deleting baths=None as all other columns are None 
filter3 = data['baths']=='None'

data=data[~filter3]
#data

In [173]:
# change all -- values to None
data = data.replace(to_replace='--',value='None')
data = data.replace(to_replace='No Data', value='None')

In [174]:
# Creating a new column based on count of nearby schools 

data['Schools']=data['Schools'].str.replace('[','')
data['Schools']=data['Schools'].str.replace(']','')
data['Nearby Schools']=data['Schools'].str.split("', '")
#data.head()
data['No. of Nearby Schools']=data['Nearby Schools'].str.len()
#data.head()
data=data.drop('Nearby Schools', axis=1)
#data.head()

In [175]:
data.to_csv('refined_data.csv', index = False, encoding = "UTF-8")